In [1]:
from pprint import pprint

input_file = "inputs/aoc_input_16.txt"
search_word = 'departure'

In [2]:
if True:
    with open(input_file) as f:
        content = f.read()
    content = content.strip().split('\n\n')
else:
    content = """
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
""".strip().split('\n\n')

In [3]:
rules_texts = content[0].split('\n')
your_ticket = [int(x) for x in content[1].split('\n')[1].split(',')]
nearby_tickets = [[int(x) for x in ticket.split(',')] for ticket in content[2].split('\n')[1:]]

In [4]:
rules = {}
for rule in rules_texts:
    rule_split = rule.split(':')
    rule_name = rule_split[0]
    
    parts = [x.strip() for x in rule_split[1].split('or')]
    parts = [{'min':int(x.split('-')[0]), 'max':int(x.split('-')[1])} for x in parts]
    
    rules[rule_name] = parts

In [5]:
def passes_rule(rule, value):
    for internal_rule in rule:
        if internal_rule['min'] <= value and value <= internal_rule['max']:
            return True
    return False

def get_wrong_tickets(nearby_tickets, rules):
    all_rules = [item for sublist in rules.values() for item in sublist]
    scanning_errors = []

    for i, ticket in enumerate(nearby_tickets):
        for ticket_value in ticket:  
            valid = False
            for rule in rules:
                if passes_rule(rules[rule], ticket_value):
                    valid = True
                    break
            if not valid:
                scanning_errors.append([i, ticket_value])
    return scanning_errors

In [6]:
wrong_tickets = get_wrong_tickets(nearby_tickets, rules)
wrong_ticket_indexes = set([x[0] for x in wrong_tickets])
sum([x[1] for x in wrong_tickets])

25916

In [7]:
valid_tickets = [nearby_tickets[i] for i in range(len(nearby_tickets)) if i not in wrong_ticket_indexes]

In [8]:
def get_possible_rule_indexes(rules, valid_tickets):
    rule_indexes = {rule_name:[] for rule_name in rules}
    for rule_name in rules:
        for column in range(len(rules)):
            all_tickets_passed = True

            for ticket in valid_tickets:
                if not passes_rule(rules[rule_name], ticket[column]):
                    all_tickets_passed = False
                    break

            if all_tickets_passed:
                rule_indexes[rule_name].append(column)
    return rule_indexes

In [9]:
def disambiguate_rules(rule_indexes):
    ordered_rules = [(x, set(y)) for x, y in sorted(rule_indexes.items(), key=lambda x: len(x[1]))]

    rules_index_map = {ordered_rules[0][0]:ordered_rules[0][1]}
    for i in range(1, len(ordered_rules)):
        rules_index_map[ordered_rules[i][0]] = ordered_rules[i][1] - ordered_rules[i-1][1]
    return {key:list(value)[0] for key, value in rules_index_map.items()}

In [10]:
rule_indexes = get_possible_rule_indexes(rules, valid_tickets)
rules_sets = disambiguate_rules(rule_indexes)

result = 1
for x in [x for x in rules_sets if x.startswith(search_word)]:
    result = result * your_ticket[rules_sets[x]]
result

2564529489989